In [1]:
import pandas as pd
import numpy as np

In [ ]:
df.read_csv('loan_train.csv')

In [ ]:
df.Dependents.value_counts()

In [ ]:
df['Dependents'].replace('3+', 3, inplace = true)

In [ ]:
ax = (df['Loan_Status'].value_counts()*100.0 /len(df)).plot(kind='bar', stacked = True, rot = 0)
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_ylabel('% Loan application')
ax.set_xlabel('Status')
ax.set_ylabel('% Customers')
ax.set_title('Application Distribution')
totals = []  
# finding the values and append to list
for i in ax.patches:
  totals.append(i.get_width())
total = sum(totals)  
for i in ax.patches:
  ax.text(i.get_x()+.15, i.get_height()-3.5, \ str(round((i.get_height()/total), 1))+'%', color='white', weight = 'bold')

In [ ]:
df.is_null.sum()

In [ ]:
# replace missing values with the mode
df['Gender'].fillna(df['Gender'].mode()[0], inplace=True)
df['Married'].fillna(df['Married'].mode()[0], inplace=True)
df['Dependents'].fillna(df['Dependents'].mode()[0], inplace=True)
df['Self_Employed'].fillna(df['Self_Employed'].mode()[0], inplace=True)
df['Credit_History'].fillna(df['Credit_History'].mode()[0], inplace=True)
df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mode()[0], inplace=True)
# replace missing values with the median value due to outliers
df['LoanAmount'].fillna(df['LoanAmount'].median(), inplace=True)
# replacing Y and N in Loan_Status variable with 1 and 0 
df['Loan_Status'].replace('N', 0, inplace=True)
df['Loan_Status'].replace('Y', 1, inplace=True)

In [ ]:
# Pearson correlation
df.corr()

In [ ]:
sns.countplot(x='Credit History', hue = 'Loan status', data = df)
plt.title('Credit History Vs loan status \ncorrlelaton: 0.54')
plt.show()

In [ ]:
# Loading test data
df1 = pd.read_csv('load_test.csv')

In [ ]:
print('Information on dataset')
df1.info()

In [ ]:
# replacing 3+ in Dependents variable with 3 
df1['Dependents'].replace('3+', 3, inplace=True)
# replace missing values in Test set with mode/median from Training set
df1['Gender'].fillna(df['Gender'].mode()[0], inplace=True)
df1['Dependents'].fillna(df['Dependents'].mode()[0], inplace=True)
df1['Self_Employed'].fillna(df['Self_Employed'].mode()[0], inplace=True)
df1['Credit_History'].fillna(df['Credit_History'].mode()[0], inplace=True)
df1['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mode()[0], inplace=True)
df1['LoanAmount'].fillna(df['LoanAmount'].median(), inplace=True)

In [ ]:
# drop Loan_ID
train = df.drop('Loan_ID', axis=1) # train
test = df1.drop('Loan_ID', axis=1) # test
# drop "Loan_Status" and assign it to target variable
X = train.drop('Loan_Status', 1)
y = train.Loan_Status
# adding dummies to the dataset
X = pd.get_dummies(X)
train = pd.get_dummies(train)
test = pd.get_dummies(test)
print(X.shape, train.shape, test.shape)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
kfold = StratifiedKFold(n_splits=5, random_state=42)
# Logistic Regression
log_reg = LogisticRegression(solver='lbfgs', max_iter=5000)
log_scores = cross_val_score(log_reg, x_train, y_train, cv=kfold)
log_reg_mean = log_scores.mean()
# SVC
svc_clf = SVC(gamma='auto')
svc_scores = cross_val_score(svc_clf, x_train, y_train, cv=kfold)
svc_mean = svc_scores.mean()
# KNearestNeighbors
knn_clf = KNeighborsClassifier()
knn_scores = cross_val_score(knn_clf, x_train, y_train, cv=kfold)
knn_mean = knn_scores.mean()
# Decision Tree
tree_clf = tree.DecisionTreeClassifier()
tree_scores = cross_val_score(tree_clf, x_train, y_train, cv=kfold)
tree_mean = tree_scores.mean()
# Gradient Boosting Classifier
grad_clf = GradientBoostingClassifier()
grad_scores = cross_val_score(grad_clf, x_train, y_train, cv=kfold)
grad_mean = grad_scores.mean()
# Random Forest Classifier
rand_clf = RandomForestClassifier(n_estimators=100)
rand_scores = cross_val_score(rand_clf, x_train, y_train, cv=kfold)
rand_mean = rand_scores.mean()
# NeuralNet Classifier
neural_clf = MLPClassifier(alpha=1)
neural_scores = cross_val_score(neural_clf, x_train, y_train, cv=kfold)
neural_mean = neural_scores.mean()
# Naives Bayes
nav_clf = GaussianNB()
nav_scores = cross_val_score(nav_clf, x_train, y_train, cv=kfold)
nav_mean = neural_scores.mean()
# Create a Dataframe with the results.
d = {'Classifiers': ['Logistic Reg.', 'SVC', 'KNN', 'Dec Tree', 'Grad B CLF', 'Rand FC', 'Neural Classifier', 'Naives Bayes'],
'Crossval Mean Scores': [log_reg_mean, svc_mean, knn_mean, tree_mean, grad_mean, rand_mean, neural_mean, nav_mean]}
result_df = pd.DataFrame(data=d)
# Log Reg performed best
result_df = result_df.sort_values(by=['Crossval Mean Scores'], ascending=False)
print(result_df)

In [ ]:
# Random Forest Classifier
rand_clf = RandomForestClassifier(n_estimators=100).fit(x_train, y_train)
# get importance
plt.barh(X.columns, rand_clf.feature_importances_)
plt.show()

In [ ]:
# Cross validate log reg classifier
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_vald_predict(log_reg, x_train, y_train, cv = kfold)